In [1]:
import sys
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import yfinance as yf
import datetime
from datetime import timedelta, date
sys.version

'3.10.1 (v3.10.1:2cd268a3a9, Dec  6 2021, 14:28:59) [Clang 13.0.0 (clang-1300.0.29.3)]'

In [2]:
%load_ext autoreload 
%autoreload 2

In [3]:
sp500_wiki_page = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies' 
sp500_ticker_csv = "data/sp500_ticker_sector.csv"

nas100_wiki_page = 'https://en.wikipedia.org/wiki/Nasdaq-100'
nas100_ticker_csv = "data/nasdaq100_ticker_sector.csv"

combined_data_csv = "data/combined_data.csv"

combined_tickers_csv = "data/combined_ticker_sector.csv"

update_log_csv = "data/update_log.csv"
#records_df = pd.read_csv(update_log_csv)

#historical_data = records_df.iloc[1,1]
updated_data  = "data/combined_data_"+ date.today().strftime('%Y-%m-%d') + ".csv"



start_default = datetime.datetime(2015, 1, 1)
# end = datetime.datetime(2023, 9, 8)

In [4]:
etf_port = ['TBIL', 'TFLO','FXC', 'IAK', 'INDA', 'EWJV','USO', 'NLR',
                'AMLP', 'UAE', 'URA', 'SMIN', 'URA', 'EWJ', 'XLE',
                'PFIX', 'INDY', 'GLD', 'PSCE', 'AAAU', 'XOP', 'UGA', 'BDRY', 'CTA', 'CYA']

stock_port = ['COST','DIS','WMT','NEM', 'MELI','VVV','DKNG', 'ATVI','MNSO','V','PCT',
             'MNST','FL','MTCH','CCL','PRGO','NYT','PLBY','TCS','YUMC','REAL','EDU','MSOS']

sectors = ['XLY','XLF','XLV','XLC','XLK','XLP','XLI','XLB','XLE','XLRE','XLU','^SPX']


In [5]:
def yahoo_last_day(ticker='GOOGL', period='1wk', interval='1d', 
                   observation='Close'):
    """Gets last date of avalible data from yahoo finance.
    
        Parameters
        ----------
        ticker : str, not required
            default = 'GOOGL'
        period : str, not required
            default = '1wk'
        interval : str, not required
            default = '1d'
        observation : str, not required
            default = 'Close'
            
        Returns
        -------
        last_day : string
            last date avalible from yahoo
        """  
    ticker = yf.Ticker(ticker)
    ticker_history = ticker.history(period, interval)
    sf = ticker_history[observation]
    df = pd.DataFrame({'Date':sf.index})
    dates = df['Date'].tolist() 
    last_day = dates

    return last_day


def get_start_end(df):
    """Determines if new data is available and calculates starting
    and ending dates for getting data from yahoo finance. 
    (if no new data available, returns same values for start and end) 
                                                                       
        Parameters
        ----------
        df : dataframe, required
            dataframe of historical ticker price data
        
        Returns
        -------
        start : string
            date string of starting date
        end : string
            date string of ending date
        """  
    
    last_day = df.index.max() 
    last_date = yahoo_last_day()
    if last_day == last_date[-1].strftime('%Y-%m-%d'):
        start = last_day
        end = last_day
    else: 
        date_1 = datetime.datetime.strptime(last_day, "%Y-%m-%d")
        start = date_1 + datetime.timedelta(days=1)
        today = date.today().strftime('%Y-%m-%d')
        end = datetime.datetime.strptime(today, "%Y-%m-%d") 
        
    
    return (start, end)

def get_ticker_data(wiki_page, table_location):
    """Gets Wiki page of tickes for SP500 and NAS100.
    
        Parameters
        ----------
        table_location : int, required
            location of table on wiki page
        Returns
        -------
        table ; dataframe
        a data frame of the Ticker information
        """
    table=pd.read_html(wiki_page)
    ticker_df = table[table_location]
    return ticker_df

def get_ticker_list(df, column_name):
    """Reformats ticker names for yahoo,
    returns df and list with corrected names
                                                                                        
        Parameters
        ----------
        df : dataframe, required
            dataframe containing tickers
        column_name : string, required
            name of the column tickers are located
        
        Returns
        -------
        df : dataframe
            original datframe with corrected ticker names
        tickers : list(str)
            list of tickers from dataframe
        """
    
    for x, ticker in enumerate(list(df[column_name])):
        if ticker.rfind('.'):
            df.loc[x,column_name] = ticker.replace(".", "-")
    tickers = list(df[column_name])
    return (df, tickers)

def save_data(df, filename, index=False):
    """Saves dataframe at defined path/name.csv. 
    
        Parameters
        ----------
        df : dataframe, required
            dataframe to be saved
        filename : string, required
            path, name and ext(.csv) of the file to be saved
        Index : logical, not required 
            Default - False
            
        Returns
        -------
        df : dataframe
            original datframe with corrected ticker names
        tickers : list(str)
            list of tickers from dataframe
        """
    
    df.to_csv(filename,index=index)

def check_for_data(df_to_read, wiki_page, table, column_name):
    """Checks if file exists, if not gets ticker data and saves file
        Used only for retriving SP500 and NAS100 data from wiki.
    
        Parameters
        ----------
        df_to_read : string, required
            read/save location (path/name.csv)
        wiki_page : string, required
            wiki page html
        table : int, required
            location of table on wiki page
        column_name : string, required
            name of the column tickers are located in the table
            
        Returns
        -------
        ticker_df
            original datframe with corrected ticker names
        ticker_list
            list of tickers from dataframe
        """
    
    path = Path(df_to_read)
    if (True!= path.is_file()):
        ticker_df = get_ticker_data(wiki_page, table)
        ticker_df, ticker_list = get_ticker_list(ticker_df,column_name )
        save_data(ticker_df, df_to_read)
        
    else:
        ticker_df = pd.read_csv(df_to_read)
        ticker_df, ticker_list = get_ticker_list(ticker_df,column_name )
        
    return ticker_df, ticker_list  
    
def update_stock_data(tickers, hist_df, start, end, file_name):
    """Gets data from yahoo finance if it is available
    
        Parameters
        ----------
        tickers : list(str), required
            list of tickers to get yahoo data
        hist_df : dataframe, required
            stored data to be updated
        start : str, required
            start date
        end : string, required
            end date
            
        Returns
        -------
        hist_df : dataframe
            dataframe of ticker data (updated if data is available)
        
        """
    if end > start:
        print('Get Data')
        stock_prices = yf.download(tickers, start=start, end=end)
        stock_prices.index = stock_prices.index.strftime('%Y-%m-%d')
        current_stock_prices = pd.concat([hist_df, stock_prices])
        current_stock_prices.to_csv(file_name)
        return current_stock_prices
    else:
        
        print(f"Stock Prices are up to date {end}")
        return  hist_df

In [6]:
# Get main Ticker data
sp500_ticker_df, sp500_ticker_list  = check_for_data(sp500_ticker_csv,
                                            sp500_wiki_page, 0, 'Symbol')
nas100_ticker_df, nas100_ticker_list = check_for_data(nas100_ticker_csv,
                                            nas100_wiki_page, 4, 'Ticker')                                                                                                                                                    
# Create combined ticker list of SP500 and NAS100
SP500_NAS100_tkrs = sp500_ticker_list + \
                   list(set(nas100_ticker_list).difference(sp500_ticker_list)) 


In [7]:
def get_combined_tickers(SP500_NAS100=SP500_NAS100_tkrs, my_stocks=stock_port,
                         my_etfs=etf_port, secttor_port=sectors):                       
    """compiles list of tickers from all ticker list sources
    
        Parameters
        ----------
        SP500_NAS100 : list(str), not required
            SP500 and NAS100 tickers unique values
            default = SP500_NAS100_tkrs : list(str)
        my_stocks : list(str), not required
            Portfolio stock tickers 
            default = stock_port : list(str)
        my_etfs : list(str), not required
            Portfolio etf tickers 
            default = etf_port : list(str)    
        sector_port : list(str), not required
            Portfolio of sector etf tickers 
            default = sectors : list(str)     
            
        Returns
        -------
        all_tickers : list(str)
            list of unique tickers from all lists
        missing_tickers : list(str)
            list of missing tickers from historical lists
        """                     
    
    missing_stocks = list(set(my_stocks).difference(SP500_NAS100)) 
    missing_portfolio  = list(set(my_etfs).difference(SP500_NAS100))  
    missing_sectors  = list(set(sectors).difference(SP500_NAS100))  
    missing_tickers = missing_stocks + missing_portfolio + missing_sectors
    all_tickers = list(set(SP500_NAS100 + missing_tickers))
    
    return all_tickers, missing_tickers   


def update_tickers(combined_tickers_csv, combined_tickers=SP500_NAS100_tkrs,
                   column_name='Symbol'):
    """Checks if file exists, if not gets ticker data and saves file
    
        Parameters
        ----------
        combined_tickers_csv : string, required
            read/save location (path/name.csv)
        combined_tickers : list(str), not required
            list of SP500 and NAS100 unique tickers 
            Default = SP500_NAS100
            
        Returns
        -------
        all_ticker_df
            original datframe with corrected ticker names
        all_tickers
            list of tickers from dataframe
        missing_tickers
        
        """
    
    path = Path(combined_tickers_csv)  # check if file exists
    if (True!= path.is_file()):        # create file if not
        all_tickers, missing_tickers = get_combined_tickers( )
        all_ticker_df = pd.DataFrame(all_tickers, columns = [column_name])
        save_data(all_ticker_df, combined_tickers_csv)
            
    else: # else get historical tickers from file
        all_ticker_df = pd.read_csv(combined_tickers_csv)
        all_ticker_df, all_ticker_lst = get_ticker_list(all_ticker_df, column_name)                                                                  
        all_tickers, missing_tickers = get_combined_tickers(all_ticker_lst)
        
        if missing_tickers !=[]:
            new_list =  missing_tickers + all_tickers
            new_list_df = pd.DataFrame(new_list, columns = [column_name])
            save_data(new_list_df, combined_tickers_csv)                                                                              
    return all_ticker_df, all_tickers, missing_tickers
                                                                                

In [8]:
#all_ticker_df, ticker_list, new_tickers = update_tickers(combined_tickers_csv)

In [9]:
'''s = ['CTA']
p=ticker_list
s in new_tickers

print(list(set(s).difference(p)) )
print(s)

'''

"s = ['CTA']\np=ticker_list\ns in new_tickers\n\nprint(list(set(s).difference(p)) )\nprint(s)\n\n"

In [11]:
path = Path(update_log_csv)
all_ticker_df, ticker_list, new_tickers = update_tickers(combined_tickers_csv)
if (True!= path.is_file()):
    
    end = datetime.datetime.strptime(date.today().strftime('%Y-%m-%d'), "%Y-%m-%d") 
    hist_stock_prices = pd.DataFrame()
    latest_prices = update_stock_data(ticker_list, hist_stock_prices,
                                      start_default, end, updated_data)
   
    last_record = end
    log={
        'last_update' : last_record,
        'file_name' : updated_data,
    }
    df = pd.DataFrame(list(log.items()))
    df.to_csv(update_log_csv, index=False)

    
    
else:
    records_df = pd.read_csv(update_log_csv)
    historical_data = records_df.iloc[1,1]
    today = datetime.datetime.today()
    hist_stock_prices = pd.read_csv(historical_data, header=[0, 1], low_memory=False, index_col=0)
    start, end = get_start_end(hist_stock_prices)
    
    if end.strftime('%Y-%m-%d') == today.strftime('%Y-%m-%d'):
        #if (today.strftime('%H') > str(20)):
        latest_prices = update_stock_data(ticker_list, hist_stock_prices, start, end, updated_data)      
        ##############
        update_log_csv = "data/update_log.csv"
        last_record = latest_prices.index[-1]
        log={
            'last_update' : last_record,
            'file_name' : updated_data,
        }
        df = pd.DataFrame(list(log.items()))
        df.to_csv(update_log_csv, index=False)
        
        #else:
            #print('Historical data availible after 9:00pm')
        
    else:
        latest_prices = update_stock_data(ticker_list, hist_stock_prices, start, end, updated_data)      
        ##############
        update_log_csv = "data/update_log.csv"
        last_record = latest_prices.index[-1]
        log={
            'last_update' : last_record,
            'file_name' : updated_data,
        }
        df = pd.DataFrame(list(log.items()))
        df.to_csv(update_log_csv, index=False)


    

Get Data
[*********************100%%**********************]  569 of 569 completed


In [ ]:
start

In [ ]:
datetime.datetime.today().strftime('%H')

In [ ]:
today = datetime.datetime.today()
(end.strftime('%Y-%m-%d') == today.strftime('%Y-%m-%d')) and (today.strftime('%H') > str(17))

In [ ]:
(today.strftime('%H') > str(17))

In [ ]:
'''def new_tkr_data(new_tickers,df_to_store, df_path):
    pass'''

In [ ]:
'''all_ticker_df, ticker_list, new_tickers = update_tickers(combined_tickers_csv)

if new_tickers !=[]:
 
    start = datetime.datetime(2015, 1, 1).strftime('%Y-%m-%d')
    today_date = datetime.datetime.strptime(date.today().strftime('%Y-%m-%d'), "%Y-%m-%d") 
    new_stocks = yf.download(new_tickers, start=start, end=today_date)
    idx = latest_prices.index.intersection(new_stocks.index)
    df = new_stocks.loc[idx].join(latest_prices.loc[idx]).sort_index(level=0, axis=1)
    df.to_csv(updated_data)
    print('New Tickers were added:' )
    print(new_tickers)
else:
    print('no new stocks added')'''

In [ ]:


# latest_prices[('Adj Close', 'CTA')]


In [ ]:
'''s = 'CYA'
s in ticker_list'''

In [ ]:
# hist_stock_prices[('Adj Close','CYA')]


In [ ]:
yahoo_last_day()